# The War Room

Simulation & training notebook for the **War on Art** auction strategy engine.

Use this notebook to simulate battles before connecting to the real Catawiki MCP.

In [ ]:
# SIMULATION CELL
import sys; sys.path.insert(0, '..')
from strategy.inputs import AuctionParams
from strategy.engine import WarOnArtEngine
from strategy.learner import ReinforcementLearner

# 1. Configure Codruta's Intent
params = AuctionParams(
    lot_id="12345",
    lot_url="https://catawiki...",
    max_budget=500,     # She wants to spend max \u20ac500
    greediness=85       # She REALLY wants this (High Greed)
)

print(f"\U0001f4b0 Visible Budget: \u20ac{params.max_budget}")
print(f"\U0001f6e1\ufe0f True Ceiling (Ozymandias Offset): \u20ac{params.true_ceiling}")

# 2. Initialize Engine
learner = ReinforcementLearner()
engine = WarOnArtEngine(params, learner)

# 3. Simulate a Scenario: \"The Dead Zone\"
# Opponent bid \u20ac480, 10 seconds left, We are NOT leading
current_bid = 480
time_left = 10
is_leading = False

action, amount, reason = engine.evaluate_state(current_bid, time_left, is_leading)

print(f"\n\u26a1 TACTICAL DECISION: {action}")
print(f"\U0001f4b6 Bid Amount: \u20ac{amount}")
print(f"\U0001f4dd Reasoning: {reason}")

---
## Multi-Scenario Battle Simulator

Run the engine through all key phases: Opening, Skirmish, Battlefield, Dead Zone.

In [ ]:
import pandas as pd

scenarios = [
    {"name": "Opening (Day 2)",     "bid": 100, "time": 86400*5, "leader": False},
    {"name": "Skirmish (30 min)",   "bid": 250, "time": 1800,   "leader": False},
    {"name": "Skirmish (leader)",   "bid": 250, "time": 1800,   "leader": True},
    {"name": "Battlefield (45s)",   "bid": 350, "time": 45,     "leader": False},
    {"name": "Battlefield (leader)","bid": 350, "time": 30,     "leader": True},
    {"name": "Dead Zone (12s)",     "bid": 400, "time": 12,     "leader": False},
    {"name": "Dead Zone (8s)",      "bid": 450, "time": 8,      "leader": False},
    {"name": "Over Budget",         "bid": 490, "time": 5,      "leader": False},
]

results = []
for s in scenarios:
    action, amount, reason = engine.evaluate_state(s["bid"], s["time"], s["leader"])
    results.append({
        "Scenario": s["name"],
        "Current Bid": f"\u20ac{s['bid']}",
        "Time Left": f"{s['time']}s",
        "Leader?": s["leader"],
        "Action": action,
        "Bid Amount": f"\u20ac{amount:.0f}" if amount > 0 else "-",
        "Reason": reason,
    })

df = pd.DataFrame(results)
df.style.set_properties(**{'text-align': 'left'})

---
## Learner Statistics & Self-Tutoring Analysis

In [ ]:
stats = learner.get_stats()
print("=== Learner Brain ===")
print(f"  Q-Table:          {stats['q_table']}")
print(f"  Epsilon:          {stats['epsilon']:.3f}")
print(f"  Total Auctions:   {stats['total_auctions']}")
print(f"  Win Rate:         {stats['win_rate']:.1%}")
print(f"  Avg Savings:      {stats['avg_savings']:.1%}")
print(f"  Preferred Action: {stats['preferred_action']}")

---
## Simulate Learner Self-Tutoring Over 50 Auctions

Observe how Q-values evolve and strategy adapts.

In [ ]:
import random
import numpy as np

# Fresh learner for simulation
sim_learner = ReinforcementLearner.__new__(ReinforcementLearner)
sim_learner.brain_path = '/dev/null'
sim_learner.history_path = '/dev/null'
sim_learner.q_table = {"ENGAGE": 0.5, "INTIMIDATE": 0.5}
sim_learner.history = []
sim_learner.actions = ["ENGAGE", "INTIMIDATE"]
sim_learner.last_action = None
sim_learner._session_actions = []
sim_learner.epsilon = 0.3
sim_learner.learning_rate = 0.1
sim_learner.discount_factor = 0.95

q_history = {"ENGAGE": [], "INTIMIDATE": [], "epsilon": []}

for i in range(50):
    # Simulate: INTIMIDATE wins 60% and gets better prices
    action = sim_learner.get_preferred_action()
    sim_learner.record_action(action)
    
    if action == "INTIMIDATE":
        won = random.random() < 0.65
        ratio = random.uniform(0.5, 0.85) if won else random.uniform(0.9, 1.1)
    else:
        won = random.random() < 0.45
        ratio = random.uniform(0.7, 0.95) if won else random.uniform(0.85, 1.0)
    
    sim_learner.update_strategy(won, ratio)
    q_history["ENGAGE"].append(sim_learner.q_table["ENGAGE"])
    q_history["INTIMIDATE"].append(sim_learner.q_table["INTIMIDATE"])
    q_history["epsilon"].append(sim_learner.epsilon)

# Plot
try:
    import matplotlib.pyplot as plt
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    ax1.plot(q_history['ENGAGE'], label='ENGAGE', linewidth=2)
    ax1.plot(q_history['INTIMIDATE'], label='INTIMIDATE', linewidth=2)
    ax1.set_title('Q-Value Evolution Over 50 Auctions')
    ax1.set_xlabel('Auction #')
    ax1.set_ylabel('Q-Value')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(q_history['epsilon'], color='orange', linewidth=2)
    ax2.set_title('Exploration Rate (Epsilon) Decay')
    ax2.set_xlabel('Auction #')
    ax2.set_ylabel('Epsilon')
    ax2.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
except ImportError:
    print("Install matplotlib for charts: pip install matplotlib")
    for k, v in q_history.items():
        print(f"{k}: {[round(x, 3) for x in v[-10:]]}")

---
## Live Auction Control Panel

Connect to Catawiki and run the full autonomous loop.

In [ ]:
import asyncio
from catawiki.browser import CatawikiBrowser
from catawiki.monitor import AuctionMonitor

# ============================
# CODRUTA: CONFIGURE YOUR BID
# ============================
LOT_ID    = "12345"                                    # <-- Catawiki lot number
LOT_URL   = "https://www.catawiki.com/l/12345"         # <-- Full lot URL
BUDGET    = 500                                         # <-- Max EUR
GREEDINESS = 85                                         # <-- 0-100 (100 = whatever it takes)

params = AuctionParams(
    lot_id=LOT_ID,
    lot_url=LOT_URL,
    max_budget=BUDGET,
    greediness=GREEDINESS,
)

print(f"Lot: {LOT_ID}")
print(f"Budget: \u20ac{BUDGET} -> True Ceiling: \u20ac{params.true_ceiling}")
print(f"Greediness: {GREEDINESS}%")
print(f"Total cost at max budget: \u20ac{params.calculate_total_acquisition_cost(BUDGET):.2f}")

In [ ]:
# Step 1: Login to Catawiki (opens a browser window)
# Run this ONCE -- after login, cookies are saved for future sessions.

browser = CatawikiBrowser(headless=False)

async def do_login():
    await browser.launch()
    await browser.login_interactive()
    print("Logged in. Cookies saved.")

await do_login()

In [ ]:
# Step 2: Launch Autopilot
# The monitor will poll the auction and execute the engine's commands.

learner = ReinforcementLearner()
monitor = AuctionMonitor(
    params=params,
    browser=browser,
    learner=learner,
    poll_interval=2.0,
    log_callback=print,  # Print battle log to notebook output
)

print("Starting autopilot...")
battle_log = await monitor.run()
print(f"\nBattle complete. {len(battle_log)} log entries.")

In [ ]:
# Step 3: Post-Battle Debrief
print("=== Battle Log ===")
for entry in battle_log:
    print(entry)

print("\n=== Updated Learner Stats ===")
for k, v in learner.get_stats().items():
    print(f"  {k}: {v}")

In [ ]:
# Cleanup
await browser.close()
print("Browser closed.")